In [9]:
# Colab 相关设置项
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/CLRS/CLRS_notes"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/CLRS/CLRS_notes


In [0]:
%%capture
%run "第 12 章：二叉搜索树.ipynb"  # 导入 BinarySearchTree

## 13.1 红黑树的性质

#### 红黑树的定义及性质

- 红黑树是许多平衡搜索树中的一种，可以保证在最坏的情况下，基本动态操作的时间复杂度为 $O(lgn)$
  - 红黑树是一棵二叉搜索树，其在每个结点上增加一个存储位来表示结点的颜色。其确保没有一条路径会比期它路径长出 2 倍，因而近似
  是平衡的
  - 红黑树的每个结点包括 5 个属性： $color, key, left, right$ 和 $p$，如果一个节点没有子结点或父结点，则该结点相应的
  指针属性值为 $\mathrm{NIL}$
  - 这些 $\mathrm{NIL}$ 可被看做指向二叉搜索树叶结点的指针，即视为**外部节点**，而将带有关键字的结点视为树的**内部节点**
  - 红黑树是满足下面 $5$ 条**红黑性质**的二叉搜索树
    1. 每个结点或是红色或是黑色
    2. 根节点是黑色的
    3. 每个叶节点(外部节点) 是黑色的
    4. 如果一个结点是红色的，则它的两个子结点都是黑色的
      - 即一个红色结点的父结点是黑色的
      - 即一条简单的路径上，不可能同时出现 2 个红色结点
    5. 对每个结点，从该结点至其所有后代叶结点的简单路径上，均包含相同数目的黑色结点，由此可定义黑高的概念
      - 黑高(black height) 的定义
        - 从某个结点 $x$ 出发（不含该结点）到达一个叶结点的任意一条简单路径上的黑色结点个数称为该结点的黑高，记为 
        $bh(x)$

#### 红黑树的示意图

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216145017.png width=800>

##### 哨兵 $T.nil$

- 使用哨兵结点 $T.nil$ 来替代 $\mathrm{NIL}$，，即可以用其来代表所有的叶节点和根节点的父节点，哨兵除了颜色需要为黑色外，
其余的属性并不重要，但在程序中为了方便，可以将它们设置为特定的值
- 示意图
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216145122.png width=750>

##### 实际表示时，也可以省略掉叶结点

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216145209.png width=800>

#### 引理 13.1
- 一棵有 $n$ 个内部节点的红黑树的高度至多为 $2lg(n+1)$

- 证明：
  - 先证明以任一节点 $x$ 为根的子树中至少包含 $2^{bh(x)}-1$ 个内部结点
    - 可采用数归纳法证明，当 $bh(x)==0$ 时，则 $x=T.nil$,树高为 $0$ ，内部节点为 0，符合要求
    - 归纳步骤
      - 对于黑高为$bh(x)$ 的结点，其左右子结点的黑高为 $bh(x)$ 或 $bh(x)-1$，取决于$x$ 结点的颜色，由归纳法可得以 
      $x$ 为根的子树，至少包含 $(2^{bh(x)-1} - 1) + (2^{bh(x)-1} - 1) + 1 = 2^{bh(x)} - 1$ 个内部节点
  - 设树高为 $h$，则由红黑树的性质4可知，从根节点到叶结点，黑高至少为 $h/2$，可得：
    - $bh(x) \ge \frac{h}{2}$， 即 $n \ge 2^{h/2} - 1$
    - 由此可得 $h \leq 2lg(n+1)$

- 由此引理可知，红黑树上的动态集合操作 $SEARCH, MINIMUM, MAXIMUM, SUCCESSOR$ 和 $PREDECESSOR$ 可以在 $O(lgn)$ 的时间
内执行完成


#### 代码实现

###### 红黑树节点的代码实现

In [0]:
class RBNode():
  R = False
  B = True

  def __init__(self, left=None, right=None, parent=None, color=R, key=None):
    self.left = left
    self.right = right
    self.parent = parent
    self.color = color
    self.key = key
  
  def __repr__(self):
    key = self.key if self.key is not None else 'None'
    color = 31 if self.color == RBNode.R else 37  # 为了对比明显，用白色表示黑色
    return '\033[0;{}m{}\033[0m'.format(color, key)
  
  def __eq__(self, other):
    return self.key == None if other is None else self is other  # 为了兼容二叉搜索树中的相关函数

In [12]:
a = RBNode(key = 66)
a

66

In [13]:
b = RBNode(key = 66, color=RBNode.B)
b

66

###### 红黑树代码的部分实现

In [0]:
class RBTree(BinarySearchTree):
  NIL = RBNode(color=RBNode.B) # 黑色的 T.Nil 结点

  def __init__(self, root=NIL):
    self.root = root

  def _binary_tree_insert(self, z):
    """普通二叉树插入，不考虑结点的颜色是否符合要求"""
    super().insert(z)
    z.left = RBTree.NIL
    z.right = RBTree.NIL
    if self.root == z:
      z.parent = RBTree.NIL


In [17]:
rbt = RBTree()
all_nodes = [26, 17, 41, 14, 21, 30, 47, 10, 16, 19, 23, 28, 38, 7, 12, 15, 20, 35, 39, 3]
black_nodes = [26, 41, 14, 21, 47, 16, 19, 23, 28, 38, 7, 12]
for key in all_nodes:
  rbt._binary_tree_insert(RBNode(key=key))
for key in black_nodes:
  node = rbt.search(rbt.root, key)
  node.color = RBNode.B
rbt


|  |  47

|  41

|  |  |  |  39

|  |  |  38

|  |  |  |  35

|  |  30

|  |  |  28

26

|  |  |  23

|  |  21

|  |  |  |  20

|  |  |  19

|  17

|  |  |  16

|  |  |  |  15

|  |  14

|  |  |  |  12

|  |  |  10

|  |  |  |  7

|  |  |  |  |  3

## 13.2 旋转

- 红黑树的插入和删除操作，需要修改指针结构才能保证红黑树的性质。而为了保持二叉搜索树的性质，指针的修改需要通过**旋转**来完成

- 左旋与右旋
  - 示意图
    - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216152802.png width=650>
  - 左旋需要假设节点的右子节点不为 $T.nil$，右旋需要假设节点的左子节点不为 $T.nil$

#### 代码实现

##### 左旋操作

In [0]:
def left_rotate(self, x):
  """左旋操作，需要确保 $x.right 存在$"""
  y = x.right
  x.right = y.left
  if y.left != RBTree.NIL:
    y.left.parent = x
  y.left = x
  y.parent = x.parent
  y.left = x
  
  if x.parent == RBTree.NIL:
    self.root = y
  elif x.parent.left == x:
    x.parent.left = y
  else:
    x.parent.right = y

  x.parent = y


RBTree.left_rotate = left_rotate

In [25]:
rbt = RBTree()
black_nodes = [7, 4, 11, 3, 6, 9, 18, 2, 14, 19, 12, 17, 22, 20]
for key in black_nodes:
  rbt._binary_tree_insert(RBNode(key=key, color=RBNode.B))
print("原始的红黑树为：\n{}".format(rbt))

i = 11
rbt.left_rotate(rbt.search(rbt.root, i))
print("右旋结点{}后，红黑树为：\n{}".format(i, rbt))


原始的红黑树为：

|  |  |  |  22

|  |  |  |  |  20

|  |  |  19

|  |  18

|  |  |  |  17

|  |  |  14

|  |  |  |  12

|  11

|  |  9

7

|  |  6

|  4

|  |  3

|  |  |  2

右旋结点11后，红黑树为：

|  |  |  22

|  |  |  |  20

|  |  19

|  18

|  |  |  |  17

|  |  |  14

|  |  |  |  12

|  |  11

|  |  |  9

7

|  |  6

|  4

|  |  3

|  |  |  2



###### 示意图

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216154956.png width=600>

##### 右旋操作

In [0]:
def right_rotate(self, x):
  """右旋操作，前提是 $x.left$ 存在"""
  y = x.left
  x.left = y.right
  if y.right != RBTree.NIL:
    y.right.parent = x
  
  y.parent = x.parent
  y.right = x

  if x.parent == RBTree.NIL:
    self.root = y
  elif x.parent.left == x:
    x.parent.left = y
  else:
    x.parent.right = y

  x.parent = y

RBTree.right_rotate = right_rotate

In [28]:
print("原始的红黑树为：\n{}".format(rbt))

i = 18
rbt.right_rotate(rbt.search(rbt.root, i))
print("右旋结点{}后，红黑树为：\n{}".format(i, rbt))

原始的红黑树为：

|  |  |  22

|  |  |  |  20

|  |  19

|  18

|  |  |  |  17

|  |  |  14

|  |  |  |  12

|  |  11

|  |  |  9

7

|  |  6

|  4

|  |  3

|  |  |  2

右旋结点18后，红黑树为：

|  |  |  |  22

|  |  |  |  |  20

|  |  |  19

|  |  18

|  |  |  |  17

|  |  |  14

|  |  |  |  12

|  11

|  |  9

7

|  |  6

|  4

|  |  3

|  |  |  2



## 13.3 插入

- 假设要插入的结点为 $z$，可以先将 $z$ 按照满二叉搜索树性质的方法将 $z$ 插入二叉树 $T$ 中，并将 $z.colort$ 设置为红色，
然后调用 $\mathrm{RB-INSERT-FIXUP}$ 来保持红黑树的性质

#### $\mathrm{RB-INSERT-FIXUP}$

- 如果 $z$ 为根节点，需要将其颜色设为黑色
- 其它情况下，只有当 $z.parent$ 为红色时，才会违反性质，共分为三种情况
  - $z.parent$ 是 $z.parent.paent$ 的左孩子还是右孩子的情况基本对称，后续分析时假设其为左孩子

##### 情况1： $z$ 的叔节点 $y$ 是红色

- 此时更改相应节点的颜色后将 $z$ 上移即可
- 示意图
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216161343.png width=650>

##### 情况2: $z$ 的叔节点是黑色且 $z$ 是一个右孩子

- 此种情况，通过旋转操作，即可将 $z$ 变为左孩子，情况 2 即可转变为情况 3
- 旋转的目的是为了使 $z, z.parent$ 与 $z.parent.parent$ 在一条直线上

##### 情况3： $z$ 的叔节点是红色且 $z$ 是一个左孩子

- 通过旋转操作，即可使得 $T$ 满足红黑树的所有性质要求
- 情况 2 和情况 3 的示意图
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216162316.png width=650>

#### 分析

- 时间复杂度为 $O(lgn)$，因为 $\mathrm{RB-INSERT-FIXUP}$ 会沿着树上升，最高到根节点，而且所做的旋转操作最多只有 2 次，因此时间复杂度为
$O(h)$， 即 $O(lgn)$

#### 代码实现

In [0]:
def insert(self, z):
  """红黑树的插入操作，直接调用二叉树的插入操作即可，不过要将相就的 None 替换为 nil"""
  super(RBTree, self).insert(z)
  if self.root == z:
    z.parent = RBTree.NIL
  z.left = RBTree.NIL
  z.right = RBTree.NIL
  z.color = RBNode.R

  self.insert_fixup(z)

RBTree.insert = insert

In [0]:
def insert_fixup(self, z):
  while z.parent.color == RBNode.R:
    if z.parent == z.parent.parent.left:
      y = z.parent.parent.right
      if y.color == RBNode.R:  # case 1
        z.parent.color = RBNode.B
        y.color = RBNode.B
        z.parent.parent.color = RBNode.R
        z = z.parent.parent
      else:
        if z.parent.right == z: # case 2
          z = z.parent
          self.left_rotate(z)
        z.parent.color = RBNode.B
        z.parent.parent.color = RBNode.R
        self.right_rotate(z.parent.parent)
    else:
      y = z.parent.parent.left
      if y.color == RBNode.R:  # case 1
        z.parent.color = RBNode.B
        y.color = RBNode.B
        z.parent.parent.color = RBNode.R
        z = z.parent.parent
      else:
        if z.parent.left == z: # case 2
          z = z.parent
          self.right_rotate(z)
        z.parent.color = RBNode.B
        z.parent.parent.color = RBNode.R
        self.left_rotate(z.parent.parent)
      

  self.root.color = RBNode.B

RBTree.insert_fixup = insert_fixup

In [52]:
rbt = RBTree()
all_nodes = [11, 2, 14, 1, 7, 15, 5, 8]
black_nodes = [11, 14, 1, 7]
for key in all_nodes:
  rbt._binary_tree_insert(RBNode(key=key))
for key in black_nodes:
  node = rbt.search(rbt.root, key)
  node.color = RBNode.B
print("原始红黑树为：\n{}".format(rbt))
i = 4
rbt.insert(RBNode(key=i))
print("插入节点 {} 后， 红黑树为：\n{}".format(i, rbt))

原始红黑树为：

|  |  15

|  14

11

|  |  |  8

|  |  7

|  |  |  5

|  2

|  |  1

插入节点 4 后， 红黑树为：

|  |  |  15

|  |  14

|  11

|  |  8

7

|  |  5

|  |  |  4

|  2

|  |  1



## 13.4 删除

- 删除一个结点 $z$, 需要移动另外一个节点 $y$ 到 $z$ 的位置,$y$ 可能为 $z$
- 为了尽可能的保持红黑树的性质，需要将 $y$ 的颜色设置为 $z$ 的颜色
- 如果移动或删除的 $y$ 是黑色结点，为了维持性质4， $y$ 移动后，在之前 $y$ 位置处的结点的子节点 $x$ 会加上一层额外的黑色，
即双重黑色，或者红黑色，需要针对此种情况对红黑树的性质进行修复

#### $\mathrm{RB-DELETE-FIXUP}$

- 进入 $\mathrm{RB-DELETE-FIXUP}$ 的前提是 $y$ 为黑色
- 如果 $x$ 为红色，则真接将其置为黑色即可
- 如果 $x$ 为黑色，则分为四种情况
  - $x$ 是其父亲的左子节点和右子节点，是两种基本对称的情况，所以只研究 $x$ 是其父新的左子节点即可

##### 情况1： $x$ 的兄弟节点 $w$ 是红色的

- 通过颜色变换和右旋操作，将其转换为情况2 ~ 情况4，即 $w$ 是黑色的
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216173336.png width=600>

##### 情况2： $w$ 是黑色的，且 $w$ 的两个子节点也是黑色的

- 此种情况，可以将 $x.parent$ 加上一层额外的黑色，同时将 $w$ 置为红色
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216173425.png width=600> 

##### 情况3：$w$ 是黑色的，且 $w$ 的左孩子是红色，右孩子是黑色

- 通右旋操作，并更新 $w$ 节点，将 $w$ 的右孩子变为红色，转换为情况4
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216173728.png width=600>

##### 情况4： $w$ 是黑色的，且 $w$ 的右孩子是红色的

- 将 $x.parent$ 改为黑色，并对 $x.parent$ 执行左旋, 此时即可以去掉 $x$ 的双重黑色
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216174226.png width=600>


#### 代码实现

In [0]:
def transplant(self, u, v):
  """在红黑树是移动子树"""
  if u.parent == RBTree.NIL:
    self.root = v
  elif u.parent.left == u:
    u.parent.left = v
  else:
    u.parent.right = v
  v.parent = u.parent

RBTree.transplant = transplant

In [0]:
def delete(self, z):
  """删除某个节点，同时保证红黑树的性质"""
  y = z
  y_original_color = y.color
  if z.left == RBTree.NIL:
    x = z.right
    self.transplant(z, z.right)
  elif z.right == RBTree.NIL:
    x = z.left
    self.transplant(z, z.left)
  else:
    y = self.minimum(z.right)
    y_original_color = y.color
    x = y.right
    if y.parent == z:
      x.parent = y  # 此时 x 可能为 NIL，所以需要对其 parent 特殊赋值，其它的赋值均在 transplant 中完成
    else:
      self.transplant(y, y.right)
      y.right = z.right
      y.right.parent = y
    self.transplant(z, y)
    y.left = z.left
    y.left.parent = y
    y.color = z.color
  if y_original_color == RBNode.B:
    self.delete_fixup(x)

RBTree.delete = delete

In [0]:
def delete_fixup(self, x):
  while x != self.root and x.color == RBNode.B:
    if x == x.parent.left:
      w = x.parent.right
      if w.color == RBNode.R:  # case 1
        x.parent.color = RBNode.R
        w.color = RBNode.B
        self.left_rotate(x.parent)
      elif w.left.color == RBNode.B and w.right.color == RBNode.B:  # case 2
        w.color = RBNode.R
        x = x.parent
      else:
        if w.right.color == RBNode.B:  # case 3
          w.left.color = RBNode.B
          w.color = RBNode.R
          self.right_rotate(w)
          w = x.parent.right
        # case 4
        w.color = x.parent.color
        x.parent.color = RBNode.B
        w.right.color = RBNode.B
        self.left_rotate(x.parent)
        x = self.root # 让下一次循环退出循环，同时确保根为黑色
    else:
      w = x.parent.left
      if w.color == RBNode.R:  # case 1
        x.parent.color = RBNode.R
        w.color = RBNode.B
        self.right_rotate(x.parent)
      elif w.left.color == RBNode.B and w.right.color == RBNode.B:  # case 2
        w.color = RBNode.R
        x = x.parent
      else:
        if w.left.color == RBNode.B:  # case 3
          w.right.color = RBNode.B
          w.color = RBNode.R
          self.left_rotate(w)
          w = x.parent.left
        # case 4
        w.color = x.parent.color
        x.parent.color = RBNode.B
        w.left.color = RBNode.B
        self.right_rotate(x.parent)
        x = self.root # 让下一次循环退出循环，同时确保根为黑色
      

  x.color = RBNode.B #针对红黑色的情况


RBTree.delete_fixup = delete_fixup

In [107]:
rbt = RBTree()
for key in [41, 38, 31, 12, 19, 8]:
  rbt.insert(RBNode(key=key))

print("原始的红黑树为：\n{}".format(rbt))

for key in [8, 12, 19, 31, 38, 41]:
  rbt.delete(rbt.search(rbt.root, key))
  print("删除 {} 后，红黑树为：\n{}".format(key, rbt))

原始的红黑树为：

|  41

38

|  |  31

|  19

|  |  12

|  |  |  8

删除 8 后，红黑树为：

|  41

38

|  |  31

|  19

|  |  12

删除 12 后，红黑树为：

|  41

38

|  |  31

|  19

删除 19 后，红黑树为：

|  41

38

|  31

删除 31 后，红黑树为：

|  41

38

删除 38 后，红黑树为：

41

删除 41 后，红黑树为：
None
